<a href="https://colab.research.google.com/github/dillonkoh89/GA-Capstone/blob/main/Capstone_Object_Detection_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow==2.4.3
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 394.5 MB 386 bytes/s 
     |████████████████████████████████| 3.8 MB 43.5 MB/s 
     |████████████████████████████████| 462 kB 34.1 MB/s 
     |████████████████████████████████| 2.9 MB 37.4 MB/s 
     |████████████████████████████████| 591 kB 5.4 MB/s 
     |████████████████████████████████| 840 kB 28.9 MB/s 
     |████████████████████████████████| 213 kB 46.1 MB/s 
     |████████████████████████████████| 120 kB 43.3 MB/s 
     |████████████████████████████████| 6.3 MB 34.3 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 
     |████████████████████████████████| 103 kB 47.0 MB/s 
     |████████████████████████████████| 1.1 MB 31.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.9 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 30.3 MB/s 
     |████████████████████████████████| 99 kB 8.8 MB/s 
     |███████████

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.3 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive/')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/drive/My\ Drive/ /mydrive

#list contents in your drive
!ls /mydrive

Mounted at /content/drive/
'20211012 - GraphExercise.gdoc'
'20211014 - SpatialExercise (1).gsheet'
'20211014 - SpatialExercise.gsheet'
'20211022 - Test.gsheet'
'20211106 - Image Classification.gslides'
'Capstone - Model Maker Object Detection.ipynb'
'Capstone Project Proposal.gslides'
'Classification of Subreddit Posts.gslides'
 classification.zip
'Colab Notebooks'
'Copy of Architecture Design Development PowerPoint Template #1386.gslides'
 customTF1
 dad.docx
 DSIF2_Project4_Nov2.gslides
 HR2002.docx
 log
'My Drive'
 TFlite
 training
'Untitled document.gdoc'
'Untitled presentation - Pie chart 1.gsheet'
'Your big idea.gslides'


In [ ]:
%cd /mydrive/
# Walk through the TFlite data directory and list number of files
for dirpath, dirnames, filenames in os.walk("TFlite"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

/content/drive/My Drive
There are 5 directories and 0 images in 'TFlite'.
There are 0 directories and 100 images in 'TFlite/images'.
There are 0 directories and 100 images in 'TFlite/annotations'.
There are 2 directories and 0 images in 'TFlite/train'.
There are 0 directories and 80 images in 'TFlite/train/images'.
There are 0 directories and 80 images in 'TFlite/train/annotations'.
There are 2 directories and 0 images in 'TFlite/test'.
There are 0 directories and 4 images in 'TFlite/test/annotations'.
There are 0 directories and 4 images in 'TFlite/test/images'.
There are 2 directories and 0 images in 'TFlite/val'.
There are 0 directories and 16 images in 'TFlite/val/images'.
There are 0 directories and 16 images in 'TFlite/val/annotations'.


In [ ]:
#spec = model_spec.get('efficientdet_lite0')
spec = object_detector.EfficientDetLite0Spec(model_dir='log')

In [ ]:
#setting the path for creating the data
train_image_dir = '/mydrive/TFlite/train/images'
train_annot_dir = '/mydrive/TFlite/train/annotations'

test_image_dir = '/mydrive/TFlite/test/images'
test_annot_dir = '/mydrive/TFlite/test/annotations'

val_image_dir = '/mydrive/TFlite/val/images'
val_annot_dir = '/mydrive/TFlite/val/annotations'

#creating the train and test data 
train_data = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annot_dir, label_map={1: "helmet", 2: "person", 3: 'safety vest'})
test_data = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_annot_dir, label_map={1: "helmet", 2: "person", 3: 'safety vest'})
val_data = object_detector.DataLoader.from_pascal_voc(val_image_dir, val_annot_dir, label_map={1: "helmet", 2: "person", 3: 'safety vest'})

In [ ]:
model = object_detector.create(train_data, model_spec=spec, epochs=50, batch_size=8, train_whole_model=True, validation_data=val_data)


Epoch 1/50
10/10 [==============================] - 59s 1s/step - det_loss: 1.7718 - cls_loss: 1.1320 - box_loss: 0.0128 - reg_l2_loss: 0.0632 - loss: 1.8350 - learning_rate: 0.0090 - gradient_norm: 0.9643 - val_det_loss: 1.7188 - val_cls_loss: 1.1082 - val_box_loss: 0.0122 - val_reg_l2_loss: 0.0632 - val_loss: 1.7820
Epoch 2/50
10/10 [==============================] - 5s 528ms/step - det_loss: 1.6585 - cls_loss: 1.0956 - box_loss: 0.0113 - reg_l2_loss: 0.0632 - loss: 1.7217 - learning_rate: 0.0100 - gradient_norm: 1.1753 - val_det_loss: 1.6300 - val_cls_loss: 1.0454 - val_box_loss: 0.0117 - val_reg_l2_loss: 0.0632 - val_loss: 1.6932
Epoch 3/50
10/10 [==============================] - 5s 534ms/step - det_loss: 1.5627 - cls_loss: 1.0282 - box_loss: 0.0107 - reg_l2_loss: 0.0632 - loss: 1.6258 - learning_rate: 0.0099 - gradient_norm: 1.4539 - val_det_loss: 1.4455 - val_cls_loss: 0.8734 - val_box_loss: 0.0114 - val_reg_l2_loss: 0.0632 - val_loss: 1.5086
Epoch 4/50
10/10 [==================

In [ ]:
model.evaluate(test_data)

1/1 [==============================] - 5s 5s/step



{'AP': 0.38992572,
 'AP50': 0.7553967,
 'AP75': 0.3445807,
 'AP_/helmet': 0.17909949,
 'AP_/person': 0.5845127,
 'AP_/safety vest': 0.40616494,
 'APl': -1.0,
 'APm': 0.5308107,
 'APs': 0.2919463,
 'ARl': -1.0,
 'ARm': 0.604386,
 'ARmax1': 0.12621832,
 'ARmax10': 0.4374269,
 'ARmax100': 0.47690058,
 'ARs': 0.36666667}

In [ ]:
# export saved model folder and labels.txt
model.export(export_dir='/mydrive/TFlite/colab_models/tflite1/', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])

In [ ]:
model.export(export_dir='/mydrive/TFlite/colab_models/tflite2/')

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

4/4 [==============================] - 11s 3s/step



{'AP': 0.3265737,
 'AP50': 0.69554955,
 'AP75': 0.32773086,
 'AP_/helmet': 0.14936766,
 'AP_/person': 0.45932788,
 'AP_/safety vest': 0.3710256,
 'APl': -1.0,
 'APm': 0.4415116,
 'APs': 0.25595856,
 'ARl': -1.0,
 'ARm': 0.50292397,
 'ARmax1': 0.111988306,
 'ARmax10': 0.38070175,
 'ARmax100': 0.3877193,
 'ARs': 0.315}

In [ ]:
model.summary()

Model: ""
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  3234464   
_________________________________________________________________
class_net/class-predict (Sep multiple                  2331      
_________________________________________________________________
box_net/box-predict (Separab multiple                  2916      
Total params: 3,239,711
Trainable params: 3,192,575
Non-trainable params: 47,136
_________________________________________________________________
